In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import src.data_utils as data_utils
import src.tree_build as dt_tree
import src.heurestics as dt_heur
import src.tree_utils as tree_utils
import src.dt_testing_utils as dt_testing
from pprint import pprint

import numpy as np
import pandas as pd

In [3]:
adult_dataset_path = 'data/adult.data'
adult_dataset_labels = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hourse-per-week','native-country','CLASS']
adult_dataset = data_utils.preapare_dataset(adult_dataset_path, adult_dataset_labels)
adult_dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hourse-per-week,native-country,CLASS
0,32,Federal-gov,504951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,50,United-States,<=50K
1,34,Private,107793,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,Germany,>50K
2,31,Local-gov,50649,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,50,United-States,<=50K
3,22,Private,71379,Some-college,10,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,35,United-States,<=50K
4,73,Private,336007,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,2246,40,United-States,>50K


In [4]:
adult_dataset = adult_dataset[['age','workclass','education','education-num','marital-status','occupation','relationship','race','sex','hourse-per-week','CLASS']]
adult_dataset.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hourse-per-week,CLASS
0,32,Federal-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,50,<=50K
1,34,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,40,>50K
2,31,Local-gov,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,50,<=50K
3,22,Private,Some-college,10,Never-married,Prof-specialty,Not-in-family,White,Female,35,<=50K
4,73,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,40,>50K


In [5]:
"""
Example of building of the decision tree
"""
adult_training, adult_testing = data_utils.data_split(adult_dataset, 20)

adult_tree = dt_tree.build_tree_generic(dt_heur.information_gain_ratio,adult_training)
pprint(adult_tree)
# tree_accuracy_adult_training = tree_utils.tree_accuracy(adult_training, adult_tree)
# tree_accuracy_adult_testing = tree_utils.tree_accuracy(adult_testing, adult_tree)
# print("Tree accuracy for testing data {0}% and for training {1}%\n".format(tree_accuracy_adult_testing,tree_accuracy_adult_training))

{<hourse-per-week(0.37)>: >=1.5[6509] <1.5[3] : {'<1.5': ' <=50K',
                                                 '>=1.5': {<age(0.37)>: >=60.5[478] <60.5[6031] : {'<60.5': {<hourse-per-week(0.37)>: >=3.5[6027] <3.5[4] : {'<3.5': ' '
                                                                                                                                                                     '<=50K',
                                                                                                                                                             '>=3.5': {<hourse-per-week(0.37)>: >=4.5[6017] <4.5[10] : {'<4.5': ' '
                                                                                                                                                                                                                                '<=50K',
                                                                                                                                 

In [6]:
tree_accuracy_adult_training = tree_utils.tree_accuracy(adult_training, adult_tree)
tree_accuracy_adult_testing = tree_utils.tree_accuracy(adult_testing, adult_tree)
print("Tree accuracy for testing data {0:.2f}% and for training {1:.2f}%\n".format(tree_accuracy_adult_testing,tree_accuracy_adult_training))

Tree accuracy for testing data 76.30% and for training 77.15%



In [7]:
car_dataset_path = 'data/car.data'
car_dataset_labels = ['buying','maint','doors','persons','lug_boot','safety','CLASS']
car_dataset = data_utils.preapare_dataset(car_dataset_path, car_dataset_labels)
car_dataset.head()

,buying,maint,doors,persons,lug_boot,safety,CLASS
0,vhigh,low,4,2,med,med,unacc
1,high,med,2,2,small,high,unacc
2,low,low,4,more,med,high,vgood
3,vhigh,low,3,4,big,med,acc
4,low,vhigh,3,2,big,low,unacc


In [8]:
cmc_dataset_path = 'data/cmc.data'
cmc_dataset_labels = ['age','education','husbandEducation','noChildren','religion','isWorking','husbandOccupation','livingStandard', 'mediaExposure', 'CLASS']
cmc_dataset = data_utils.preapare_dataset(cmc_dataset_path, cmc_dataset_labels)
cmc_dataset.head()

,age,education,husbandEducation,noChildren,religion,isWorking,husbandOccupation,livingStandard,mediaExposure,CLASS
0,41,3,3,5,1,1,1,4,0,2
1,24,3,4,4,1,1,3,1,0,3
2,30,4,3,4,1,1,3,3,0,3
3,44,2,3,1,0,1,2,4,0,2
4,36,2,1,6,1,1,2,1,0,2


In [9]:
cancer_dataset_path = 'data/breast-cancer.data'
cancer_dataset_labels = ['CLASS','age','menopause','tumor-size','inv-nodes','node-caps','deg-malig','breast', 'breast-quad', 'irradiat']
cancer_dataset = data_utils.preapare_dataset(cancer_dataset_path, cancer_dataset_labels)
cancer_dataset = cancer_dataset[['age','menopause','tumor-size','inv-nodes', 'node-caps','deg-malig','breast', 'breast-quad', 'irradiat', 'CLASS']]
cancer_dataset.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,CLASS
0,40-49,premeno,25-29,0-2,no,2,right,left_low,no,recurrence-events
1,40-49,premeno,25-29,0-2,no,2,left,left_up,yes,no-recurrence-events
2,40-49,premeno,45-49,0-2,no,2,left,left_low,yes,no-recurrence-events
3,50-59,ge40,35-39,0-2,no,2,left,left_low,no,recurrence-events
4,50-59,ge40,20-24,0-2,no,1,right,left_low,no,no-recurrence-events


In [10]:
"""
Example of building of the decision tree
"""
car_training, car_testing = data_utils.data_split(car_dataset, 70)

car_tree= dt_tree.build_tree_generic(dt_heur.distance_measure,car_training)
pprint(car_tree)
tree_accuracy_car_training = tree_utils.tree_accuracy(car_training, car_tree)
tree_accuracy_car_testing = tree_utils.tree_accuracy(car_testing, car_tree)
print("Tree accuracy for testing data {0}% and for training {1}%\n".format(tree_accuracy_car_testing,tree_accuracy_car_training))

{<doors(1.00)>: 2[306] 3[298] 4[300] 5more[305] : {'2': {<maint(0.98)>: high[74] low[81] med[80] vhigh[71] : {'high': {<lug_boot(0.97)>: big[23] med[25] small[26] : {'big': {<buying(0.88)>: high[6] low[7] med[4] vhigh[6] : {'high': 'unacc',
                                                                                                                                                                                                                                'low': 'unacc',
                                                                                                                                                                                                                                'med': 'unacc',
                                                                                                                                                                                                                                'vhigh': 'unacc'}},
                                   

In [11]:
"""
Second example of building of the decision tree
"""
cmc_training, cmc_testing = data_utils.data_split(cmc_dataset, 70)

cmc_tree= dt_tree.build_tree_generic(dt_heur.distance_measure,cmc_training)
pprint(cmc_tree)
tree_accuracy_cmc_training = tree_utils.tree_accuracy(cmc_testing, cmc_tree)
tree_accuracy_cmc_testing = tree_utils.tree_accuracy(cmc_testing, cmc_tree)

print("Tree accuracy for testing data {0}% and for training {1}%\n".format(tree_accuracy_cmc_testing,tree_accuracy_cmc_training ))

{<isWorking(1.00)>: 0[267] 1[764] : {0: {<mediaExposure(0.99)>: 0[249] 1[18] : {0: {<religion(0.99)>: 0[51] 1[198] : {0: {<age(0.66)>: >=41.5[6] <41.5[45] : {'<41.5': {<noChildren(0.66)>: >=4.5[4] <4.5[41] : {'<4.5': {<husbandOccupation(0.66)>: >=1.5[32] <1.5[9] : {'<1.5': 1,
                                                                                                                                                                                                                                                                          '>=1.5': {<education(0.66)>: >=3.5[22] <3.5[10] : {'<3.5': 1,
                                                                                                                                                                                                                                                                                                                             '>=3.5': 2}}}},
                                                              

Tree accuracy for testing data 38.23529411764706% and for training 38.23529411764706%



In [12]:
print("car dataset")
dt_testing.decision_tree_testing_for_given_dataset(car_dataset)

car dataset
Testing with heuristic=gini_impurity_weighted
Accuracy training=91.90% testing=86.34%, levels=10, leafs=74, training data=50%
Accuracy training=91.12% testing=85.69%, levels=10, leafs=70, training data=60%
Accuracy training=91.40% testing=86.90%, levels=10, leafs=71, training data=70%
Accuracy training=90.59% testing=87.28%, levels=10, leafs=69, training data=80%
Testing with heuristic=info_gain
Accuracy training=91.90% testing=86.34%, levels=10, leafs=74, training data=50%
Accuracy training=91.51% testing=86.27%, levels=10, leafs=72, training data=60%
Accuracy training=91.07% testing=85.36%, levels=10, leafs=69, training data=70%
Accuracy training=90.59% testing=87.28%, levels=10, leafs=69, training data=80%
Testing with heuristic=information_gain_ratio
Accuracy training=92.82% testing=86.11%, levels=10, leafs=80, training data=50%
Accuracy training=91.70% testing=86.13%, levels=10, leafs=73, training data=60%
Accuracy training=91.40% testing=86.90%, levels=10, leafs=71, t

In [13]:
print("CMC dataset")
dt_testing.decision_tree_testing_for_given_dataset(cmc_dataset)

CMC dataset
Testing with heuristic=gini_impurity_weighted
Accuracy training=78.67% testing=46.13%, levels=28, leafs=156, training data=50%
Accuracy training=77.80% testing=48.81%, levels=34, leafs=169, training data=60%
Accuracy training=75.17% testing=48.19%, levels=30, leafs=177, training data=70%
Accuracy training=74.62% testing=50.17%, levels=30, leafs=182, training data=80%
Testing with heuristic=info_gain
Accuracy training=58.29% testing=44.50%, levels=72, leafs=194, training data=50%
Accuracy training=56.85% testing=44.75%, levels=62, leafs=198, training data=60%
Accuracy training=53.93% testing=46.15%, levels=74, leafs=198, training data=70%
Accuracy training=53.57% testing=47.46%, levels=68, leafs=202, training data=80%
Testing with heuristic=information_gain_ratio
Accuracy training=57.61% testing=42.74%, levels=72, leafs=191, training data=50%
Accuracy training=57.42% testing=44.75%, levels=64, leafs=199, training data=60%
Accuracy training=54.61% testing=46.61%, levels=74, l

In [14]:
print("cancer dataset")

# heuristics = [dt_heur.gini_impurity_weighted, dt_heur.info_gain, dt_heur.information_gain_ratio,
#                       dt_heur.distance_measure, dt_heur.j_measure, dt_heur.weight_of_evidence,
#                       dt_heur.gini_pri, dt_heur.relief, dt_heur.relevance, dt_heur.mdl_simple]
# dt_testing.decision_tree_testing_for_given_dataset(cmc_dataset, heuristics)

dt_testing.decision_tree_testing_for_given_dataset(cmc_dataset)

cancer dataset
Testing with heuristic=gini_impurity_weighted
Accuracy training=78.67% testing=46.13%, levels=28, leafs=156, training data=50%
Accuracy training=77.80% testing=48.81%, levels=34, leafs=169, training data=60%
Accuracy training=75.17% testing=48.19%, levels=30, leafs=177, training data=70%
Accuracy training=74.62% testing=50.17%, levels=30, leafs=182, training data=80%
Testing with heuristic=info_gain
Accuracy training=58.29% testing=44.50%, levels=72, leafs=194, training data=50%
Accuracy training=56.85% testing=44.75%, levels=62, leafs=198, training data=60%
Accuracy training=53.93% testing=46.15%, levels=74, leafs=198, training data=70%
Accuracy training=53.57% testing=47.46%, levels=68, leafs=202, training data=80%
Testing with heuristic=information_gain_ratio
Accuracy training=57.61% testing=42.74%, levels=72, leafs=191, training data=50%
Accuracy training=57.42% testing=44.75%, levels=64, leafs=199, training data=60%
Accuracy training=54.61% testing=46.61%, levels=74

In [15]:
print("adult dataset")
# dt_testing.decision_tree_testing_for_given_dataset(adult_dataset)
heuristics = [dt_heur.gini_impurity_weighted]
dt_testing.decision_tree_testing_for_given_dataset(adult_dataset, heuristics)

adult dataset
Testing with heuristic=gini_impurity_weighted
Accuracy training=84.02% testing=82.20%, levels=22, leafs=422, training data=50%
Accuracy training=83.76% testing=82.46%, levels=20, leafs=425, training data=60%
Accuracy training=83.64% testing=82.64%, levels=18, leafs=423, training data=70%
Accuracy training=83.69% testing=82.45%, levels=18, leafs=439, training data=80%


In [29]:
print("adult dataset")
heuristics = [dt_heur.info_gain]
dt_testing.decision_tree_testing_for_given_dataset(adult_dataset, heuristics)

adult dataset
Testing with heuristic=info_gain
Accuracy training=76.81% testing=76.56%, levels=228, leafs=412, training data=50%
Accuracy training=76.98% testing=76.61%, levels=232, leafs=411, training data=60%
Accuracy training=77.08% testing=76.61%, levels=230, leafs=464, training data=70%
Accuracy training=77.10% testing=76.82%, levels=226, leafs=489, training data=80%


In [ ]:
print("adult dataset")
heuristics = [dt_heur.information_gain_ratio]
dt_testing.decision_tree_testing_for_given_dataset(adult_dataset, heuristics)

adult dataset
Testing with heuristic=information_gain_ratio
Accuracy training=76.81% testing=76.64%, levels=228, leafs=403, training data=50%
Accuracy training=76.99% testing=76.60%, levels=232, leafs=410, training data=60%


In [18]:
print("adult dataset")
heuristics = [dt_heur.distance_measure]
dt_testing.decision_tree_testing_for_given_dataset(adult_dataset, heuristics)

adult dataset
Testing with heuristic=distance_measure
Accuracy training=79.55% testing=78.58%, levels=12, leafs=371, training data=50%
Accuracy training=79.54% testing=78.76%, levels=14, leafs=382, training data=60%
Accuracy training=79.61% testing=78.83%, levels=14, leafs=408, training data=70%
Accuracy training=79.61% testing=79.01%, levels=14, leafs=430, training data=80%


In [21]:
# Information based heuristics - info_gain, information_gain_ratio, j_measure, weight_of_evidence
# To check for information based heuristics - Why so many levels? very long execution of algorithm
# heuristics = [dt_heur.j_measure]
# dt_testing.decision_tree_testing_for_given_dataset(adult_dataset, heuristics)

In [22]:
# heuristics = [dt_heur.weight_of_evidence]
# dt_testing.decision_tree_testing_for_given_dataset(adult_dataset, heuristics)

In [26]:
heuristics =[dt_heur.gini_pri]
dt_testing.decision_tree_testing_for_given_dataset(adult_dataset, heuristics)

Testing with heuristic=gini_pri
Accuracy training=81.39% testing=80.94%, levels=12, leafs=332, training data=50%
Accuracy training=81.27% testing=81.10%, levels=14, leafs=347, training data=60%
Accuracy training=81.33% testing=81.13%, levels=14, leafs=358, training data=70%
Accuracy training=81.48% testing=80.84%, levels=12, leafs=353, training data=80%


In [28]:
heuristics = [dt_heur.relief]
dt_testing.decision_tree_testing_for_given_dataset(adult_dataset, heuristics)

Testing with heuristic=relief
Accuracy training=82.07% testing=81.09%, levels=14, leafs=433, training data=50%
Accuracy training=81.87% testing=81.39%, levels=14, leafs=450, training data=60%
Accuracy training=81.86% testing=81.57%, levels=14, leafs=458, training data=70%
Accuracy training=81.94% testing=81.47%, levels=14, leafs=460, training data=80%


In [27]:
heuristics = [dt_heur.relevance]
dt_testing.decision_tree_testing_for_given_dataset(adult_dataset, heuristics)

Testing with heuristic=relevance
Accuracy training=79.41% testing=79.02%, levels=20, leafs=193, training data=50%
Accuracy training=79.24% testing=79.18%, levels=16, leafs=214, training data=60%
Accuracy training=79.14% testing=79.16%, levels=16, leafs=197, training data=70%
Accuracy training=79.10% testing=79.06%, levels=22, leafs=218, training data=80%


In [25]:
heuristics = [dt_heur.mdl_simple]
dt_testing.decision_tree_testing_for_given_dataset(adult_dataset, heuristics)

Testing with heuristic=mdl_simple
Accuracy training=83.37% testing=82.27%, levels=10, leafs=482, training data=50%
Accuracy training=83.17% testing=82.38%, levels=10, leafs=479, training data=60%
Accuracy training=83.20% testing=82.23%, levels=10, leafs=482, training data=70%
Accuracy training=83.36% testing=82.24%, levels=12, leafs=536, training data=80%
